# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [1]:
!pip install -r ../requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 128.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 140.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 121.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 28.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 165.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 122.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 173.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.6/156.6 kB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [27]:
!pip install seaborn
!pip install --upgrade pip

!pip install numpy 
!pip install Flask 
!pip install request
!pip install render_template pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.4/232.4 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 34.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request
ERROR: Could not find a version that satisfies the requirement render_template (from versions: none)
ERROR: No matching distribution found for render_template


In [28]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

from flask import Flask, request, render_template
import pickle

In [4]:
url = "https://raw.githubusercontent.com/4GeeksAcademy/random-forest-project-tutorial/main/titanic_train.csv"
df = pd.read_csv(url)

In [5]:
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
364,365,0,3,"O'Brien, Mr. Thomas",male,NaN,1,0,370365,15.500,NaN,Q
259,260,1,2,"Parrish, Mrs. (Lutie Davis)",female,50.0,0,1,230433,26.000,NaN,S
327,328,1,2,"Ball, Mrs. (Ada E Hall)",female,36.0,0,0,28551,13.000,D,S
610,611,0,3,"Andersson, Mrs. Anders Johan (Alfrida Konstant...",female,39.0,1,5,347082,31.275,NaN,S
68,69,1,3,"Andersson, Miss. Erna Alexandra",female,17.0,4,2,3101281,7.925,NaN,S


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
df["Survived"].value_counts()
#la muestra no esta desbalanceada

0    549
1    342
Name: Survived, dtype: int64

In [9]:
df=df.drop(columns=["Name"])
#name no tiene valor predictivo la elimino

In [10]:
df["Sex"]=df["Sex"].map({"male":1,"female":0})
df["Sex"]

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [11]:
df["Ticket"].nunique()

681

In [36]:
df=df.drop(columns=["Ticket"])
# no aporta a la predicción

KeyError: "['Ticket'] not found in axis"

In [41]:
df=df.drop(columns=["Cabin"])
df=df.drop(columns=["PassengerId"])
df=df.drop(columns=["SibSp"])

# no aporta a la predicción

In [13]:
df["Embarked"].value_counts()
df["Embarked"].nunique()

3

In [14]:
df["Embarked"]=df["Embarked"].map({"S":2,"C":1,"Q":0})
df["Embarked"]

0      2.0
1      1.0
2      2.0
3      2.0
4      2.0
      ... 
886    2.0
887    2.0
888    2.0
889    1.0
890    0.0
Name: Embarked, Length: 891, dtype: float64

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    float64
dtypes: float64(3), int64(4)
memory usage: 48.9 KB


In [16]:
df["Age"].mean()
# analizo las edades

29.69911764705882

In [34]:
df["Age"][np.isnan(df["Age"])]=df["Age"].mean()
#sino tiene edad lo completo con la media

/tmp/ipykernel_2278/2469530718.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Age"][np.isnan(df["Age"])]=df["Age"].mean()


In [18]:
df["Embarked"].value_counts()

2.0    644
1.0    168
0.0     77
Name: Embarked, dtype: int64

In [33]:
df['Embarked'][np.isnan(df['Embarked'])]=2
# completo con 2 los valores en 0

/tmp/ipykernel_2278/151368785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Embarked'][np.isnan(df['Embarked'])]=2


In [39]:
df.sample(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
389,390,1,2,0,17.0,0,0,12.0000,1.0
777,778,1,3,0,5.0,0,0,12.4750,2.0
53,54,1,2,0,29.0,1,0,26.0000,2.0
394,395,1,3,0,24.0,0,2,16.7000,2.0
569,570,1,3,1,32.0,0,0,7.8542,2.0


In [44]:
X=df.drop(columns=['Survived'])
y = df[['Survived']]

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=520, test_size=0.2)

In [46]:
modelo = RandomForestClassifier(n_estimators=20, random_state=42, max_depth=5)
modelo.fit(X_train, y_train)
#Uso RandomForestClassifier

/tmp/ipykernel_2278/2951520533.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo.fit(X_train, y_train)


RandomForestClassifier(max_depth=5, n_estimators=20, random_state=42)

In [47]:
y_train_pred=modelo.predict(X_train)
y_test_pred=modelo.predict(X_test)

In [48]:
print(metrics.classification_report(y_pred=y_train_pred, y_true=y_train))

              precision    recall  f1-score   support

           0       0.83      0.97      0.90       456
           1       0.92      0.66      0.77       256

    accuracy                           0.86       712
   macro avg       0.88      0.81      0.83       712
weighted avg       0.87      0.86      0.85       712



In [49]:
print(metrics.classification_report(y_pred=y_test_pred, y_true=y_test))

              precision    recall  f1-score   support

           0       0.70      0.97      0.81        93
           1       0.94      0.56      0.70        86

    accuracy                           0.77       179
   macro avg       0.82      0.76      0.76       179
weighted avg       0.82      0.77      0.76       179



In [50]:
filename = '../models/titanic_model.pickle'
pickle.dump(modelo, open(filename, 'wb'))

In [51]:
titanic_model = pickle.load(open('../models/titanic_model.pickle', 'rb'))